In [23]:
import torch
import torch.nn.functional as F
import lpips
import numpy as np
from skimage.metrics import structural_similarity as ssim
from math import log10
from PIL import Image
import torchvision.transforms as transforms

# Initialize LPIPS model
loss_fn_alex = lpips.LPIPS(net='alex')

def image_to_tensor(image):
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    return transform(image).unsqueeze(0)

def resize_images(image1, image2):
    min_width = min(image1.width, image2.width)
    min_height = min(image1.height, image2.height)
    image1_resized = image1.resize((min_width, min_height))
    image2_resized = image2.resize((min_width, min_height))
    return image1_resized, image2_resized

def calculate_psnr(image1, image2):
    img1 = image_to_tensor(image1)
    img2 = image_to_tensor(image2)
    
    mse = F.mse_loss(img1, img2)
    if mse == 0:
        return float('inf')
    return 10 * log10(1.0 / mse.item())

def calculate_ssim(image1, image2):
    img1 = np.array(image1)
    img2 = np.array(image2)
    
    # Ensure win_size is smaller than the smallest dimension of the images
    win_size = min(img1.shape[0], img1.shape[1], 7) if min(img1.shape[0], img1.shape[1]) >= 7 else 3

    # Calculate SSIM
    return ssim(img1, img2, multichannel=True, data_range=img2.max() - img2.min(), win_size=win_size, channel_axis=-1)

def calculate_lpips(image1, image2):
    img1 = image_to_tensor(image1)
    img2 = image_to_tensor(image2)
    
    # Normalize the images to the range [-1, 1]
    img1 = (img1 * 2) - 1
    img2 = (img2 * 2) - 1

    # Calculate LPIPS
    lpips_score = loss_fn_alex(img1, img2)
    return lpips_score.item()

# Example usage
if __name__ == "__main__":
    # Load images
    #
    # Load images
    high_res_image_path = r'C:\Users\nilot\OneDrive\Desktop\IntelAi\original.png'
    super_resolved_image_path = r'C:\Users\nilot\OneDrive\Desktop\IntelAi\resolved_image.jpg'
    
    img1 = Image.open(high_res_image_path).convert('RGB')
    img2 = Image.open(super_resolved_image_path).convert('RGB')
    
    # Resize images to the same dimensions
    img1_resized, img2_resized = resize_images(img1, img2)

    psnr_value = calculate_psnr(img1_resized, img2_resized)
    print(f"PSNR: {psnr_value}")

    ssim_value = calculate_ssim(img1_resized, img2_resized)
    print(f"SSIM: {ssim_value}")

    lpips_value = calculate_lpips(img1_resized, img2_resized)
    print(f"LPIPS: {lpips_value}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\nilot\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\weights\v0.1\alex.pth
PSNR: 26.556410241208855
SSIM: 0.7690410796504582
LPIPS: 0.30649393796920776


For FPS metrics refer to the below notebook:
https://www.kaggle.com/code/che3zcake/generatormodelfps 